In [1]:
import numpy as np
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import Adam

In [2]:
# load the wine dataset
data = np.loadtxt(r"wine.data", delimiter=",")

In [3]:
# split the data into input (X) and output (Y) variables
X = data[:,1:14]
Y = data[:,0]

In [4]:
# encode the output variable as integers
encoder = LabelEncoder()
encoder.fit(Y)
Y = encoder.transform(Y)

In [5]:
# convert the output variable to a one-hot encoded binary array
Y = np_utils.to_categorical(Y)

In [6]:
# split the data into training and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33)

In [7]:
# create the model
model = Sequential()
model.add(Dense(64, input_dim=13, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(3, activation='softmax'))

In [8]:
# compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [9]:
# fit the model to the training data
model.fit(X_train, Y_train, epochs=100, batch_size=10)

Epoch 1/100
12/12 [==============================] - 0s 2ms/step - loss: 23.7993 - accuracy: 0.2269
Epoch 2/100
12/12 [==============================] - 0s 1ms/step - loss: 8.8078 - accuracy: 0.2017
Epoch 3/100
12/12 [==============================] - 0s 2ms/step - loss: 3.9707 - accuracy: 0.1933
Epoch 4/100
12/12 [==============================] - 0s 2ms/step - loss: 1.4439 - accuracy: 0.4790
Epoch 5/100
12/12 [==============================] - 0s 1ms/step - loss: 1.1724 - accuracy: 0.5378
Epoch 6/100
12/12 [==============================] - 0s 1ms/step - loss: 1.2700 - accuracy: 0.5966
Epoch 7/100
12/12 [==============================] - 0s 1ms/step - loss: 1.1025 - accuracy: 0.6050
Epoch 8/100
12/12 [==============================] - 0s 1ms/step - loss: 0.8219 - accuracy: 0.6639
Epoch 9/100
12/12 [==============================] - 0s 910us/step - loss: 0.6640 - accuracy: 0.7059
Epoch 10/100
12/12 [==============================] - 0s 2ms/step - loss: 0.6750 - accuracy: 0.6807
Epoch 

In [10]:
# evaluate the model on the test data
scores = model.evaluate(X_test, Y_test)
print("Accuracy: %.2f%%" % (scores[1]*100))

2/2 [==============================] - 0s 4ms/step - loss: 0.1880 - accuracy: 0.9492
Accuracy: 94.92%


In [11]:
def create_model(optimizer='adam', hidden_size=64):
    model = Sequential()
    model.add(Dense(units=32, activation='relu', input_shape=(13,)))
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=5, verbose=0)
param_grid = {'optimizer': ['SGD', 'RMSprop', 'Adam'], 'hidden_size': [32, 64, 128]}
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train, Y_train)

C:\Users\Chinu\AppData\Local\Temp\ipykernel_32204\835142763.py:8: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=5, verbose=0)


In [12]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
     print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.881522 using {'hidden_size': 64, 'optimizer': 'Adam'}
0.252174 (0.026709) with: {'hidden_size': 32, 'optimizer': 'SGD'}
0.814855 (0.094094) with: {'hidden_size': 32, 'optimizer': 'RMSprop'}
0.857971 (0.076813) with: {'hidden_size': 32, 'optimizer': 'Adam'}
0.252174 (0.026709) with: {'hidden_size': 64, 'optimizer': 'SGD'}
0.756159 (0.076736) with: {'hidden_size': 64, 'optimizer': 'RMSprop'}
0.881522 (0.082904) with: {'hidden_size': 64, 'optimizer': 'Adam'}
0.285507 (0.088265) with: {'hidden_size': 128, 'optimizer': 'SGD'}
0.874275 (0.058429) with: {'hidden_size': 128, 'optimizer': 'RMSprop'}
0.857971 (0.096812) with: {'hidden_size': 128, 'optimizer': 'Adam'}
